In [237]:
import pandas as pd
import numpy as np
import plotly.express as px


# Sampling Price Distribution

**Objective**: How to learn and simulate stock price

**Data**: LoB from Lobster Dataset

In [238]:
url_path = "LOBSTER/AAPL_2012-06-21_34200000_57600000_orderbook_10.csv"
lob_data = pd.read_csv(url_path, header=None)
print(lob_data.shape)
lob_data.head()


(400391, 40)


,0,1,2,3,4,5,6,7,8,9,...,30,31,32,33,34,35,36,37,38,39
0,5859400,200,5853300,18,5859800,200,5853000,150,5861000,200,...,5845300,300,5876500,1160,5843800,200,5879000,500,5842700,300
1,5859400,200,5853300,18,5859800,200,5853200,18,5861000,200,...,5846500,300,5876500,1160,5845300,300,5879000,500,5843800,200
2,5859400,200,5853300,18,5859800,200,5853200,18,5861000,200,...,5849300,300,5876500,1160,5846500,300,5879000,500,5845300,300
3,5859100,18,5853300,18,5859400,200,5853200,18,5859800,200,...,5849300,300,5873900,100,5846500,300,5876500,1160,5845300,300
4,5859100,18,5853300,18,5859200,18,5853200,18,5859400,200,...,5849300,300,5871000,10,5846500,300,5873900,100,5845300,300


In [239]:
# Preprocessing Steps
#puting label
def rename_lob_columns(data):
    cols = ['ask_price', 'ask_size', 'bid_price', 'bid_size']
    new_column_names = []
    num_levels = len(data.columns) // len(cols) # how many group of 4
    for i in range(num_levels):
        new_column_names.extend(f"{name}_{i+1}" for name in cols)

    # Rename the columns
    data.columns = new_column_names
    return data

lob_data = rename_lob_columns(lob_data)
lob_data.head()

,ask_price_1,ask_size_1,bid_price_1,bid_size_1,ask_price_2,ask_size_2,bid_price_2,bid_size_2,ask_price_3,ask_size_3,...,bid_price_8,bid_size_8,ask_price_9,ask_size_9,bid_price_9,bid_size_9,ask_price_10,ask_size_10,bid_price_10,bid_size_10
0,5859400,200,5853300,18,5859800,200,5853000,150,5861000,200,...,5845300,300,5876500,1160,5843800,200,5879000,500,5842700,300
1,5859400,200,5853300,18,5859800,200,5853200,18,5861000,200,...,5846500,300,5876500,1160,5845300,300,5879000,500,5843800,200
2,5859400,200,5853300,18,5859800,200,5853200,18,5861000,200,...,5849300,300,5876500,1160,5846500,300,5879000,500,5845300,300
3,5859100,18,5853300,18,5859400,200,5853200,18,5859800,200,...,5849300,300,5873900,100,5846500,300,5876500,1160,5845300,300
4,5859100,18,5853300,18,5859200,18,5853200,18,5859400,200,...,5849300,300,5871000,10,5846500,300,5873900,100,5845300,300


In [240]:
# Create a time column based on message table
messages_path = "LOBSTER/AAPL_2012-06-21_34200000_57600000_message_10.csv"
messages = pd.read_csv(messages_path, header=None)
messages.columns = ['Time', 'Type', 'OrderID', 'Size', 'Price', 'Direction']
messages['Direction'] = messages['Direction'].replace({-1: 'Sell limit order', 1: 'Buy limit order'})
# implement time column corresponding to the lob data
lob_data['time'] = messages['Time'] #time is in seconds

In [241]:
lob_data.head()

,ask_price_1,ask_size_1,bid_price_1,bid_size_1,ask_price_2,ask_size_2,bid_price_2,bid_size_2,ask_price_3,ask_size_3,...,bid_size_8,ask_price_9,ask_size_9,bid_price_9,bid_size_9,ask_price_10,ask_size_10,bid_price_10,bid_size_10,time
0,5859400,200,5853300,18,5859800,200,5853000,150,5861000,200,...,300,5876500,1160,5843800,200,5879000,500,5842700,300,34200.004241
1,5859400,200,5853300,18,5859800,200,5853200,18,5861000,200,...,300,5876500,1160,5845300,300,5879000,500,5843800,200,34200.004261
2,5859400,200,5853300,18,5859800,200,5853200,18,5861000,200,...,300,5876500,1160,5846500,300,5879000,500,5845300,300,34200.004447
3,5859100,18,5853300,18,5859400,200,5853200,18,5859800,200,...,300,5873900,100,5846500,300,5876500,1160,5845300,300,34200.025552
4,5859100,18,5853300,18,5859200,18,5853200,18,5859400,200,...,300,5871000,10,5846500,300,5873900,100,5845300,300,34200.025580


In [242]:
price_data = lob_data.copy()

# __Step 1__: Calculate the mid price from the average best bid and ask price

In [243]:
# Caclulate mid price from the best ask and bid price
# Note: The mid price is a snapshot of the current price of the stock (here APPLE stock)
price_data['mid_price'] = (price_data['ask_price_1'] + price_data['bid_price_1']) / 2
price_data['mid_price'] = price_data['mid_price']/10000 # to get the price in dollars
price_data.head()

,ask_price_1,ask_size_1,bid_price_1,bid_size_1,ask_price_2,ask_size_2,bid_price_2,bid_size_2,ask_price_3,ask_size_3,...,ask_price_9,ask_size_9,bid_price_9,bid_size_9,ask_price_10,ask_size_10,bid_price_10,bid_size_10,time,mid_price
0,5859400,200,5853300,18,5859800,200,5853000,150,5861000,200,...,5876500,1160,5843800,200,5879000,500,5842700,300,34200.004241,585.635
1,5859400,200,5853300,18,5859800,200,5853200,18,5861000,200,...,5876500,1160,5845300,300,5879000,500,5843800,200,34200.004261,585.635
2,5859400,200,5853300,18,5859800,200,5853200,18,5861000,200,...,5876500,1160,5846500,300,5879000,500,5845300,300,34200.004447,585.635
3,5859100,18,5853300,18,5859400,200,5853200,18,5859800,200,...,5873900,100,5846500,300,5876500,1160,5845300,300,34200.025552,585.620
4,5859100,18,5853300,18,5859200,18,5853200,18,5859400,200,...,5871000,10,5846500,300,5873900,100,5845300,300,34200.025580,585.620


# __Step 2__: Resample the prices in OHLC (open, high, low, and close price and volume for a given period) *here in minutes

_pandas docs_

ohlc method: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.core.resample.Resampler.ohlc.html



In [244]:
# Resample the prices in OHLCV in minutes
price_data['time'] = pd.to_datetime(price_data['time'], unit='s')
price_data.set_index('time', inplace=True)
price_data = price_data.resample('1min').ohlc() # resample the data in 1 minute bins
# only take the mid price
price_data = price_data['mid_price']
price_data = price_data.add_suffix('_mid_price')
price_data.head()


,open_mid_price,high_mid_price,low_mid_price,close_mid_price
time,,,,
1970-01-01 09:30:00,585.635,585.850,585.310,585.510
1970-01-01 09:31:00,585.510,585.625,584.735,585.075
1970-01-01 09:32:00,585.035,585.480,584.925,585.480
1970-01-01 09:33:00,585.480,587.065,585.295,586.865
1970-01-01 09:34:00,586.865,587.720,586.850,587.300


# __Step 3__: Work with closing_mid_price and calculate the returns (percentage of change of mid_price for each minute)

Formula: (current_close_price - previous_close_price)/previous_close_price ...

pandas function: pct_change() --> Docs:https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.pct_change.html




In [245]:
#price_data['returns_auto'] = price_data['close_mid_price'].pct_change()
price_data['returns'] = (price_data['close_mid_price'] - price_data['close_mid_price'].shift(1)) / price_data['close_mid_price'].shift(1)

price_data.head(10)

,open_mid_price,high_mid_price,low_mid_price,close_mid_price,returns
time,,,,,
1970-01-01 09:30:00,585.635,585.850,585.310,585.510,NaN
1970-01-01 09:31:00,585.510,585.625,584.735,585.075,-0.000743
1970-01-01 09:32:00,585.035,585.480,584.925,585.480,0.000692
1970-01-01 09:33:00,585.480,587.065,585.295,586.865,0.002366
1970-01-01 09:34:00,586.865,587.720,586.850,587.300,0.000741
1970-01-01 09:35:00,587.300,587.300,586.515,586.625,-0.001149
1970-01-01 09:36:00,586.630,587.485,586.580,587.475,0.001449
1970-01-01 09:37:00,587.475,587.600,587.015,587.015,-0.000783
1970-01-01 09:38:00,587.015,587.075,585.590,585.920,-0.001865


# __Step 4__: Fit a Gaussian Distribution to this return data, estimate the mean and standard deviation (using closed form estimators or numerically with maximum likelihood)

*Reminder of Gaussian Distribution*

<p align="center" >
<img src="/Users/lucazosso/Desktop/IE_Course/Term_3/Algorithmic_Trading/sampling_price_dist/normal_distribution.png" alt="Gaussiance Image" width="400"/>
</p>


→ Symmetric around the mean
→ Mean = Median = Mode
→ Asymptotic tails (no intersecation)
Closed Form method:

__Methods used to estimate the parameters of a distribution__

1. Closed Form: 
Algebric Formulas (mean, standard deviation, ect)

Maximum Likelihood method: → The goal is to find the optimal way to fit a distribution to the data.

_For Gaussian Distribution we are trying to maximize this:_
$$
f(x) = \frac{1}{\sqrt{2\pi\sigma^2}} e^{-\frac{(x-\mu)^2}{2\sigma^2}}
$$

Note: If we were to take the derivative of the formula to find the maximum we would fine x = μ, which is the mean of the distribution.




In [246]:
from scipy.stats import norm

# Use Maximum Likelihood Estimation (MLE) to estimate the mean and standard deviation
mean_returns_mle, std_returns_mle = norm.fit(price_data['returns'].dropna())
print("Estimated Mean (MLE):", mean_returns_mle)
print("Estimated Standard Deviation (MLE):", std_returns_mle)

# Use closed form method to estimate the mean and standard deviation
mean_returns_closed_form = price_data['returns'].mean()
std_returns_closed_form = price_data['returns'].std()
print("Mean (Closed Form):", mean_returns_closed_form)
print("Standard Deviation (Closed Form):", std_returns_closed_form)


Estimated Mean (MLE): -3.482843252455602e-05
Estimated Standard Deviation (MLE): 0.00047842605605002953
Mean (Closed Form): -3.482843252455602e-05
Standard Deviation (Closed Form): 0.0004790421877361734


In [247]:
import plotly.graph_objects as go
returns = price_data['returns'].dropna()
# Create the histogram of returns
hist_data = go.Histogram(x=returns, histnorm='probability density', name='Distribution of returns')

# Generate points for the Gaussian fit
x = np.linspace(min(returns), max(returns), 1000)
pdf = norm.pdf(x, mean_returns_closed_form, std_returns_closed_form)

# Create the line trace for the Gaussian fit
gaussian_fit = go.Scatter(x=x, y=pdf, mode='lines', name='Fitted Gaussian PDF', line=dict(color='red', width=2))

# Combine traces in one plot
fig_gauss = go.Figure(data=[hist_data, gaussian_fit])

# Update layout for a cleaner look
fig_gauss.update_layout(
    title='Returns Data and Fitted Gaussian Distribution - AKA Empirical Distribution',
    xaxis_title='Returns',
    yaxis_title='Probability Density',
    bargap=0.2,  # Space between bars
    template='plotly_white'  # Cleaner template
)

# Show plot
fig_gauss.show()

# Compute the excess kurtosis (normal distribution has excess kurtosis of 3)
excess_kurtosis = returns.kurtosis()
print("Excess Kurtosis:", excess_kurtosis)

# Compute the skewness (normal distribution has skewness of 0)
skewness = returns.skew()
print("Skewness:", skewness)

Excess Kurtosis: 2.45132378581814
Skewness: 0.03204291838811766


In [248]:
# Compute the Saphiro Test
# H0: The data is normally distributed
# H1: The data is not normally distributed
from scipy.stats import shapiro
shapiro_test = shapiro(returns)
print("Shapiro Test Statistic:", shapiro_test.pvalue)
if shapiro_test.pvalue < 0.05:
    print("Shapiro Test: The data is not normally distributed")

Shapiro Test Statistic: 1.7292255733991624e-06
Shapiro Test: The data is not normally distributed


In [249]:
# Plotting the returns timeseries
returns_time = px.line(price_data, x=price_data.index, y='returns', title='Returns Trend')
returns_time.update_xaxes(title_text='Time')
returns_time.update_yaxes(title_text='Returns')
returns_time.show()

__Comments__:

After fitting the normal distribution to our return data and computed some statistics we can deduce the following. Visually speaking the distribution looks symmetric around the mean and also by plotting the retruns trend it looks like we have some stationarity around the mean as well that confirms normallity. 

However, some outliers in the distribution (left + right tail) shows us that our returns distribution are fat tailed. This is confirm by the excess kurtosis (measure of the tail and how sharp the peak is) of around +2 compared to Normal. Furthermore, when computing the Saphiro-Wilk Test for normallity, the test told us to reject normality assumption.

# __Step 4__: Let's Sample

 In order to generate simulations we start with the same initial mid-price from the lobster dataset (resampled in minutes)

1. Direct Sampling from the Gaussian Distribution
2. Inverse Transform Sampling

### 1. **Direct Sampling**

We are going to use the normal distribution build around the mean and std of the returns that we computed previously.

* Fix the sample size

* Generate the sample from the Gaussian Distribution

* Start from the last price of the dataset

* Simulate the price by iterating through the gaussian_distribution and find the new price with respect to the returns

* Visualize in a graph the close price sample and our orginial

_Notes_

To find the new_price we are using the returns formula as follows:
$$
r = (P1 - P0)/P0
$$
$$
P1 = P0+P0×r=P0×(1+r)
$$
r = return
P1 = NewPrice$
P0 = PrevPrice

In [250]:
price_data.head()

,open_mid_price,high_mid_price,low_mid_price,close_mid_price,returns
time,,,,,
1970-01-01 09:30:00,585.635,585.850,585.310,585.510,NaN
1970-01-01 09:31:00,585.510,585.625,584.735,585.075,-0.000743
1970-01-01 09:32:00,585.035,585.480,584.925,585.480,0.000692
1970-01-01 09:33:00,585.480,587.065,585.295,586.865,0.002366
1970-01-01 09:34:00,586.865,587.720,586.850,587.300,0.000741


In [251]:
price_data['returns'].describe()    

count    389.000000
mean      -0.000035
std        0.000479
min       -0.001865
25%       -0.000299
50%       -0.000051
75%        0.000238
max        0.002366
Name: returns, dtype: float64

In [252]:
num_samples = 390 # CLT --> 390 minutes in a trading day
seed = 42
np.random.seed(seed)

#Generate sample directly from the gaussian distribution
sample_gaussian_returns = np.random.normal(loc=mean_returns_closed_form, scale=std_returns_closed_form, size=num_samples)
print(f"First sample return {sample_gaussian_returns[0]}")
# Simulate Stock Price trajectory
# Start from the last price in the lob dataset
start_price = price_data['close_mid_price'].iloc[0] #-1

# Initialize the simulated price list
sim_prices = [start_price]

for r in sample_gaussian_returns:
    #calculate the new price based on the previous price
    new_price = sim_prices[-1] * (1 + r)
    #append to the list
    sim_prices.append(new_price)

First sample return 0.00020311860201346524


In [253]:
fig_sim_direct_method = go.Figure()

# Add the historical price trace
fig_sim_direct_method.add_trace(go.Scatter(
    x=list(range(len(price_data['close_mid_price']))), 
    y=price_data['close_mid_price'], 
    mode='lines', 
    name='Historical Closing Prices'
))

# Add the simulated price trace
fig_sim_direct_method.add_trace(go.Scatter(
    x=list(range(len(sim_prices))), 
    y=sim_prices, 
    mode='lines', 
    name='Simulated Stock Prices'))



# Update the layout to add titles and labels
fig_sim_direct_method.update_layout(
    title='Simulated Stock Price Trajectory Using Direct Sampling',
    xaxis_title='Time Steps',
    yaxis_title='Price',
    template='plotly_dark'  # Using a dark theme for better visibility
)

# Show the interactive plot
fig_sim_direct_method.show()


In [254]:
# Stacked histogram of returns (origirnal and simulated)
fig_dist_direct_method = go.Figure()

# historical returns
fig_dist_direct_method.add_trace(go.Histogram(x=returns, histnorm='probability density', name='Historical Returns'))
#Simulated returns
fig_dist_direct_method.add_trace(go.Histogram(x=sample_gaussian_returns, histnorm='probability density', name='Simulated Returns'))

fig_dist_direct_method.update_layout(
    title='Comparison of Historical and Simulated Returns',
    xaxis_title='Returns',
    yaxis_title='Probability Density',
    barmode='overlay',  # Overlay the histograms
    bargap=0.1,  # Space between bars
    template='plotly_dark'  # Using a dark theme for better visibility
)

fig_dist_direct_method.show()

### 2. **Inverse Transform Sampling**

What is it?

Inverse Transform Sampling is a method that allows us to generate random samples from any probability distribution, given we have its Cumulative Distribution Function (CDF). This method is particularly useful because it does not require prior knowledge of how to directly sample from the distribution. It only requires the ability to compute the inverse of the CDF.

Understanding the CDF: 

The CDF of a distribution maps values from the variable's range (e.g., heights, weights, temperatures) to a probability between 0 and 1. This probability indicates the likelihood of the random variable being less than or equal to a given value. For instance, if the CDF at a certain height is 0.3, it means there is a 30% chance that a randomly selected height from the distribution is less than or equal to that height. 
$$
Fx(x) = P(X <= x)
$$

The Uniform Distribution: 

Inverse transform sampling leverages the fact that if you take a uniform random number between 0 and 1, this number can be thought of as a probability itself. 0.4 = 4% probability, ect...

Using the Inverse CDF: 

By applying the inverse of the CDF to a uniform random number, we transform this uniform probability back into a value from the original distribution. 

_For example, if we generate a random number like 0.5 from the uniform distribution and apply the inverse CDF, it will give us the median of the original distribution._

Generating Random Samples: 

You start by generating random numbers uniformly distributed between 0 and 1. Each of these numbers represents a percentile in the context of the distribution you're sampling from. Applying the inverse CDF to each number translates it back into a corresponding value from the original distribution. This is equivalent to randomly selecting a percentile and then determining which value corresponds to this percentile in the original distribution's context.

**_Example_**
Assume you have a theoretical CDF for heights where you know how to calculate the percentile for any given height.
Generate a uniform random number, say 0.25, which you interpret as the 25th percentile.
Applying the inverse CDF to 0.25 might tell us that the height corresponding to the 25th percentile is 150 cm, for example.


Step to do

* Fix the sample size

* Generate a uniform random sample (numbers between 0 and 1)

* We apply the inverse CDF of the Gaussian Distribution

* Start from the last price of the dataset

* Simulate the price by iterating through the gaussian_distribution and find the new price with respect to the returns

* Visualize in a graph the close price sample and our orginial


In [255]:
from scipy.interpolate import interp1d

#Estimate the CDF of the originral returns
data_sorted = np.sort(returns)
probs = np.linspace(start=1/len(returns), stop=1, num=len(returns)) #computing cdf

#create the inverse cdf
inverse_cdf_function = interp1d(probs, data_sorted, bounds_error=False, fill_value="extrapolate") #need to use extrapolate to get values outside the range (gaps)

#generate uniform samples (numbers from 0 to 1) using uniform distribution
num_samples = 390  # 390 minutes in a trading day
uniform_samples = np.random.uniform(0, 1, num_samples)

#Sample data from the inverse cdf
returns_sampled_inv_trans_dist = inverse_cdf_function(uniform_samples)

# Start from the last price in the lob dataset
start_price = price_data['close_mid_price'].iloc[0] #-1

# Initialize the simulated price list
sim_prices = [start_price]

for r in returns_sampled_inv_trans_dist:
    #calculate the new price based on the previous price
    new_price = sim_prices[-1] * (1 + r)
    #append to the list
    sim_prices.append(new_price)

In [256]:
# Plot simulated price trajectory
fig_sim_inv_method = go.Figure()

# Add the historical price trace
fig_sim_inv_method.add_trace(go.Scatter(
    x=list(range(len(price_data['close_mid_price']))), 
    y=price_data['close_mid_price'], 
    mode='lines', 
    name='Historical Closing Prices'
))

# Add the simulated price trace
fig_sim_inv_method.add_trace(go.Scatter(
    x=list(range(len(sim_prices))), 
    y=sim_prices, 
    mode='lines', 
    name='Simulated Stock Prices'
))

# Update the layout to add titles and labels
fig_sim_inv_method.update_layout(
    title='Simulated Stock Price Trajectory Using Inverse Transform Sampling',
    xaxis_title='Time Steps',
    yaxis_title='Price',
    template='plotly_dark'  # Using a dark theme for better visibility
)

# Show the interactive plot
fig_sim_inv_method.show()

In [257]:
# Stacked histogram of returns (origirnal and simulated)
fig_dist_inv_method = go.Figure()

# historical returns
fig_dist_inv_method.add_trace(go.Histogram(x=returns, histnorm='probability density', name='Historical Returns'))

# Simulated returns
fig_dist_inv_method.add_trace(go.Histogram(x=returns_sampled_inv_trans_dist, histnorm='probability density', name='Simulated Returns'))

fig_dist_inv_method.update_layout(
    title='Comparison of Historical and Simulated Returns',
    xaxis_title='Returns',
    yaxis_title='Probability Density',
    barmode='overlay',  # Overlay the histograms
    bargap=0.1,  # Space between bars
    template='plotly_dark'  # Using a dark theme for better visibility
)

fig_dist_inv_method.show()
                           

In [258]:
# Display the CDF of the returns
fig_cdf = go.Figure()

# Create the CDF of the historical returns
hist, bins = np.histogram(returns, bins=100, density=True)
cdf = np.cumsum(hist) * np.diff(bins)
fig_cdf.add_trace(go.Scatter(x=bins[1:], y=cdf, mode='lines', name='Historical Returns'))                          

fig_cdf.update_layout(
    title='CDF of Historical Returns',
    xaxis_title='Returns',
    yaxis_title='Cumulative Probability',
    template='plotly_dark'  # Using a dark theme for better visibility
)

fig_cdf.show()


# Comparaison & Statement

* Recalculate Mean and Standard Deviation from Sampled Returns
* Compare the statistical value

In [259]:
# Recalculate the mean and standard deviation of the simulated returns
mean_returns_simulated_direct = np.mean(sample_gaussian_returns)
std_returns_simulated_direct = np.std(sample_gaussian_returns)
excess_kurtosis_simulated_direct = pd.Series(sample_gaussian_returns).kurtosis()

print("Using Direct Sampling: \n")
print(f"Sampled Mean: {mean_returns_simulated_direct}")
print(f"Sampled Standard Deviation: {std_returns_simulated_direct}")
print(f"Sampled Excess Kurtosis: {excess_kurtosis_simulated_direct}")
print(f"Ratio between stdev from original and simulated data: {std_returns_closed_form/std_returns_simulated_direct}")

mean_returns_simulated_inv = np.mean(returns_sampled_inv_trans_dist)
std_returns_simulated_inv = np.std(returns_sampled_inv_trans_dist)
excess_kurtosis_simulated_inv = pd.Series(returns_sampled_inv_trans_dist).kurtosis()

print("\nUsing Inverse Transform Sampling: \n")
print(f"Sampled Mean: {mean_returns_simulated_inv}")
print(f"Sampled Standard Deviation: {std_returns_simulated_inv}")
print(f"Sampled Excess Kurtosis: {excess_kurtosis_simulated_inv}")
# If ratio is close to 1, then the simulation is good why: because the std deviation is the same
print(f"Ratio between stdev from original and simulated data: {std_returns_closed_form/std_returns_simulated_inv}")

# Original returns mean and std deviations
original_mean = price_data['returns'].mean()
original_std = price_data['returns'].std()

print("\nOriginal Data: \n")
print(f"Original Mean: {original_mean}")
print(f"Original Standard Deviation: {original_std}")
print(f"Original Excess Kurtosis: {excess_kurtosis}")

Using Direct Sampling: 

Sampled Mean: -2.5689246620755153e-05
Sampled Standard Deviation: 0.00045636024897332346
Sampled Excess Kurtosis: 0.560051726076451
Ratio between stdev from original and simulated data: 1.0497018283557291

Using Inverse Transform Sampling: 

Sampled Mean: -4.8291599384769616e-05
Sampled Standard Deviation: 0.000454958086662332
Sampled Excess Kurtosis: 2.148776070787756
Ratio between stdev from original and simulated data: 1.0529369666786834

Original Data: 

Original Mean: -3.482843252455602e-05
Original Standard Deviation: 0.0004790421877361734
Original Excess Kurtosis: 2.45132378581814


# Conclusion & Limitation of sampling the price

Alright so here are my Conclusions of my work:

1. To sample the price, we should take the returns (or log of the returns) are it is more stationary. (consistency nd reliability)

2. The Gaussian assumption for returns are not valid as financial data has often fat tails (outliers) + higher kurtosis value (data are more spread out). --> Can lead to underestimation of extreme risks but also there are more data that falls within 1stdev!)

3. The difference between the direct sampling method and the inverse transform sampling is that inverse transform sampling don't assumes any form of distribution -> It can adapt to any empirical distributions. Therefore we can see when running the different samples a higher kurtosis.

4. The limitations are clearly that those methodes assumes IID data (independent and identically distributed data). However, all the observations are time dependent (therefore a time series analysis could be more benefitial) but also can have autocorrelations and cluster of variabilities!. (Open Question: Those taking the past to infer the future is that accurate in finance?)


Additional Research:
- In our original data, if we compute the max returns(outliers) we see that it started at 9:30 hour of the opening of the market where often a large volume is traded. --> There should be impact on market openings!

- There seems to be a notion of volume that should be taken into account

- What happens if we try to sample two stocks?

- What if we start to sample using all the negative values first and then positive values?

# External Ressources Used


__Paper__
https://towardsdatascience.com/why-is-gaussian-the-king-of-all-distributions-c45e0fe8a6e5#:~:text=Gaussian%20Distribution%20and%20its%20key,the%20mean%20with%20asymptotic%20tails.



__Programming Documents__
https://pandas.pydata.org/docs/

https://docs.scipy.org/doc/scipy/reference/stats.html

https://plotly.com/python/histograms/#overlaid-histogram


__Videos__
https://www.youtube.com/watch?v=Dn6b9fCIUpM&t=33s

https://www.youtube.com/watch?v=xmR0uvAxWAo



# Annexes

Substract the outliers Research

In [260]:
# Take the Outliers out of the returns using IQR method
Q1 = price_data['returns'].quantile(0.25)
Q3 = price_data['returns'].quantile(0.75)
IQR = Q3 - Q1
returns_no_outliers = price_data['returns'][(price_data['returns'] > (Q1 - 1.5 * IQR)) & (price_data['returns'] < (Q3 + 1.5 * IQR))]

# Plot the empirical distribution of the returns without outliers

hist_data_no_outliers = go.Histogram(x=returns_no_outliers, histnorm='probability density', name='Distribution of returns without outliers')
fig_gauss_no_outliers = go.Figure(data=[hist_data_no_outliers, gaussian_fit])
fig_gauss_no_outliers.update_layout(
    title='Returns Data and Fitted Gaussian Distribution - AKA Empirical Distribution',
    xaxis_title='Returns',
    yaxis_title='Probability Density',
    bargap=0.2,  # Space between bars
    template='plotly_white'  # Cleaner template
)
fig_gauss_no_outliers.show()


# use vriable gaussian_fit to plot the gaussian fit 
# Generate points for the Gaussian fit

# Run the Shapiro-Wilk test on the returns without outliers
shapiro_test_no_outliers = shapiro(returns_no_outliers)
print("Shapiro Test Statistic (No Outliers):", shapiro_test_no_outliers.pvalue)
if shapiro_test_no_outliers.pvalue < 0.05:
    print("Shapiro Test (No Outliers): The data is not normally distributed")
else:
    print("Shapiro Test (No Outliers): The data is normally distributed")

excess_kurtosis_no_outliers = returns_no_outliers.kurtosis()
print("Excess Kurtosis (No Outliers):", excess_kurtosis_no_outliers)

fig_gauss.show()
    



Shapiro Test Statistic (No Outliers): 0.4690149426460266
Shapiro Test (No Outliers): The data is normally distributed
Excess Kurtosis (No Outliers): -0.007252188119625913


In [261]:
# Check at what time of the day the outliers return are happening
# Use IQR variable previously calculated and price_data table previously created

#Get outliers
outliers = ~price_data['returns'].isin(returns_no_outliers)
outliers = price_data['returns'][outliers==True]
#orders outliers by value
outliers = outliers.sort_values(ascending=False)
outliers

time
1970-01-01 09:33:00    0.002366
1970-01-01 13:42:00    0.001461
1970-01-01 09:36:00    0.001449
1970-01-01 10:11:00    0.001077
1970-01-01 10:45:00    0.001048
1970-01-01 12:53:00    0.001046
1970-01-01 14:26:00   -0.001119
1970-01-01 13:19:00   -0.001148
1970-01-01 09:35:00   -0.001149
1970-01-01 09:54:00   -0.001278
1970-01-01 09:49:00   -0.001355
1970-01-01 15:34:00   -0.001424
1970-01-01 15:19:00   -0.001559
1970-01-01 12:03:00   -0.001565
1970-01-01 09:38:00   -0.001865
1970-01-01 09:30:00         NaN
Name: returns, dtype: float64

__What happends if we start to sample using the all the negative returns first and then all the positive?

In [262]:
# What happends if we sample with the returns from smaller to larger
# Using Direct Sampling

num_samples = 390 # CLT --> 390 minutes in a trading day
seed = 42
np.random.seed(seed)

#Generate sample directly from the gaussian distribution
sample_gaussian_returns = np.random.normal(loc=mean_returns_closed_form, scale=std_returns_closed_form, size=num_samples)
print(f"First sample return {sample_gaussian_returns[0]}")

# Order samples s-l
sample_gaussian_returns = np.sort(sample_gaussian_returns)#[::-1]

# Simulate Stock Price trajectory
# Start from the last price in the lob dataset
start_price = price_data['close_mid_price'].iloc[0] #-1

# Initialize the simulated price list
sim_prices = [start_price]

for r in sample_gaussian_returns:
    #calculate the new price based on the previous price
    new_price = sim_prices[-1] * (1 + r)
    #append to the list
    sim_prices.append(new_price)

First sample return 0.00020311860201346524


In [263]:
fig_sim_direct_method_sorted = go.Figure()

# Add the historical price trace
fig_sim_direct_method_sorted.add_trace(go.Scatter(
    x=list(range(len(price_data['close_mid_price']))), 
    y=price_data['close_mid_price'], 
    mode='lines', 
    name='Historical Closing Prices'
))

# Add the simulated price trace
fig_sim_direct_method_sorted.add_trace(go.Scatter(
    x=list(range(len(sim_prices))), 
    y=sim_prices, 
    mode='lines', 
    name='Simulated Stock Prices'))



# Update the layout to add titles and labels
fig_sim_direct_method_sorted.update_layout(
    title='Simulated Stock Price Trajectory Using Direct Sampling',
    xaxis_title='Time Steps',
    yaxis_title='Price',
    template='plotly_dark'  # Using a dark theme for better visibility
)

# Show the interactive plot
fig_sim_direct_method_sorted.show()

__Impact of small negative bias in Returns on Simulated Stock Prices__

In [264]:
import numpy as np
import matplotlib.pyplot as plt

# Initial price
price = price_data['close_mid_price'].iloc[0]
# Number of periods
periods = 390
# Simulate a small negative bias in returns
#returns = np.random.normal(-0.0001, 0.01, periods)

# Simulate a small positive bias in returns
returns = np.random.normal(0.0001, 0.01, periods)

prices = [price]
for r in returns:
    price = price * (1 + r)
    prices.append(price)

# Plot the results using plotly show original and simulated price
fig_test_small_bias = go.Figure()
# Add the historical price trace
fig_test_small_bias.add_trace(go.Scatter(
    x=list(range(len(price_data['close_mid_price']))), 
    y=price_data['close_mid_price'], 
    mode='lines', 
    name='Historical Closing Prices'
))

# Add the simulated price trace
fig_test_small_bias.add_trace(go.Scatter(
    x=list(range(len(prices))), 
    y=prices, 
    mode='lines', 
    name='Simulated Stock Prices'
))

# Update the layout to add titles and labels
fig_test_small_bias.update_layout(
    title='Simulated Stock Price Trajectory with Small Negative Bias',
    xaxis_title='Time Steps',
    yaxis_title='Price',
    template='plotly_dark'  # Using a dark theme for better visibility
)

# Show the interactive plot
fig_test_small_bias.show()

# Comments:
# The variance of the price is way higher than the original data!

# Print statistics
print("Original Data: \n")
print(f"Original Returns Mean: {original_mean}")
print(f"Original Returns Standard Deviation: {original_std}")
print(f"Original Returns Excess Kurtosis: {excess_kurtosis}")

print("\nSimulated Data: \n")
print(f"Simulated Returns Mean: {np.mean(returns)}")
print(f"Simulated Returns Standard Deviation: {np.std(returns)}")
print(f"Simulated Returns Excess Kurtosis: {pd.Series(returns).kurtosis()}")

print("\n Original Price Data: \n")
print(f"Original Price Data Mean: {price_data['close_mid_price'].mean()}")
print(f"Original Price Data Standard Deviation: {price_data['close_mid_price'].std()}")

print("\n Simulated Price Data: \n")
print(f"Simulated Price Data Mean: {np.mean(prices)}")
print(f"Simulated Price Data Standard Deviation: {np.std(prices)}")






Original Data: 

Original Returns Mean: -3.482843252455602e-05
Original Returns Standard Deviation: 0.0004790421877361734
Original Returns Excess Kurtosis: 2.45132378581814

Simulated Data: 

Simulated Returns Mean: -0.0002871438049419998
Simulated Returns Standard Deviation: 0.01020698140232144
Simulated Returns Excess Kurtosis: -0.17479309204250804

 Original Price Data: 

Original Price Data Mean: 583.4769615384615
Original Price Data Standard Deviation: 2.47047029047112

 Simulated Price Data: 

Simulated Price Data Mean: 536.871465316533
Simulated Price Data Standard Deviation: 36.857598231817576
